# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
# alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19

activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14

fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10

fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [ ]:
source = pd.DataFrame({'Grupo de edad':fallecidos_etareo.tail(7)["grupo_de_edad"].values,
              'Fallecidos hasta la fecha':fallecidos_etareo.tail(7)["fallecidos"].values})

barra=alt.Chart(source).mark_bar(color='green',opacity=0.7).encode(
    x=alt.X('Grupo de edad:O'),
    y='Fallecidos hasta la fecha:Q',
    tooltip=['Grupo de edad','Fallecidos hasta la fecha']
).properties(
    title= 'Cantidad de fallecidos hasta la fecha por edad',
    width=800,
    height=800
)

text = alt.Chart(source).mark_bar().encode(
    x='Grupo de edad',
    y='Fallecidos hasta la fecha',
    tooltip=['Grupo de edad','Fallecidos hasta la fecha']
).mark_text(baseline='middle').encode(
    text='Fallecidos hasta la fecha'
) 

barra+text

**Comentarios:** A partir de la gráfica podemos decir que las personas entre 60 y 89 años son las que en mayor cantidad la enfermedad les ha afectado fatalmente, por lo que podemos reafirmar que los adultos mayores son los que están en mayor riesgo ante la enfermedad, especialmente las que se encuentran entre los 70 y 79 años.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:
comunas = activos.comuna.unique()
poblacion_comunas = []
for nom_comuna in activos.comuna.unique():
    poblacion_comunas.append(activos.loc[activos["comuna"] == nom_comuna]["poblacion"].unique()[0])
    
df = pd.concat([ pd.DataFrame({"comunas": comunas}), pd.DataFrame({"poblacion":poblacion_comunas}) ], axis=1)   

In [ ]:
puntos= alt.Chart(df).mark_circle().encode(
    x='comunas:N',
    y=alt.Y('poblacion:Q', axis=alt.Axis(title='Poblacion'))
).properties(
    width=1400,
    height=400
)

prom = alt.Chart(df).mark_rule(color='red').encode(
    y='mean(poblacion):Q',
    tooltip=alt.Tooltip('mean(poblacion):Q', title="Población nacional promedio"),
    size=alt.value(1)
).properties(
    width=1400,
    height=400
)

puntos+prom

**Comentarios:** Bajo el promedio nacional pareciera encontrarse la mayoría de las comunas y poco dispersas, sin embargo las comunas sobre el promedio es lo contrario, llegando incluso a diferencias de alrededor de 600000 habitantes.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:
def calcular_diferencia(col):
    diferencia=[0]
    indices=col.index
    for x in range(1,len(indices)):
        diferencia.append(col[indices[x]]-col[indices[x-1]])
    return diferencia

In [ ]:
total_fallecidos= pd.DataFrame(fallecidos_etareo.groupby("fecha")["fallecidos"].sum()).reset_index()

fallecidos_por_edad = alt.Chart(fallecidos_etareo).mark_area(opacity=0.7).encode(
    x=alt.X("fecha", title="Fecha"),
    y=alt.Y("fallecidos:Q", title="Fallecidos"),
    color=alt.Color("grupo_de_edad:N", title="Grupo etario", scale=alt.Scale(scheme='set1')),
    tooltip=[alt.Tooltip("grupo_de_edad", title="Grupo etario"), 
             alt.Tooltip("fecha", title="Fecha"), 
             alt.Tooltip("fallecidos", title="Fallecidos")]
).properties(
    title="Fallecidos por grupo etario",
    width=1400,
    height=600
)

fallecidos_total = alt.Chart(total_fallecidos).mark_point(opacity=0).encode(
    x=alt.X("fecha", title="Fecha"),
    y=alt.Y("fallecidos:Q", title= "Fallecidos"),
    tooltip=[alt.Tooltip("fecha", title="Fecha"), alt.Tooltip("fallecidos", title="Total fallecidos")]
)

fallecidos_por_edad+fallecidos_total

**Comentarios:** Nuevamente se confirma que los adultos mayores son los que mueren en mayor cantidad por el virus y que durante todo el periodo ha sido así, además se puede observar que desde Junio a mediados de Julio la cantidad de fallecidos total aumentó más rápido que en todo el periodo registrado, aún así la cantidad de muertos sigue en aumento.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
def calcular_diferencia(col):
    diferencia=[0]
    indices=col.index
    for x in range(1,len(indices)):
        diferencia.append(col[indices[x]]-col[indices[x-1]])
    return diferencia

In [ ]:
casos_confirmados = pd.DataFrame(confirmados.groupby("fecha")["casos_confirmados"].sum()).reset_index().assign(casos_confirmados_nuevos = (lambda df: df["casos_confirmados"].transform(lambda col: calcular_diferencia(col))))

fallecimientos = pd.DataFrame(fallecidos.drop(fallecidos[fallecidos['region']=="Total"].index, axis=0).groupby("fecha")["fallecidos"].sum()).reset_index().assign(fallecidos_nuevos = (lambda df: df["fallecidos"].transform(lambda col: calcular_diferencia(col))))

casos_activos = pd.DataFrame(activos.groupby("fecha")["casos_activos"].sum()).reset_index()

In [ ]:
#Casos confirmados
plot_confirmados = alt.Chart(casos_confirmados).mark_line().encode(
    x=alt.X("fecha:T", title= "Fecha"),
    y=alt.Y("casos_confirmados_nuevos", title= "Casos confirmados"),
    tooltip = "casos_confirmados_nuevos"
).properties(
    title="Evolución casos confirmados",
    width=1400,
    height=500
)   

#Fallecimientos
plot_fallecimientos = alt.Chart(fallecimientos).mark_line().encode(
    x=alt.X("fecha:T", title= "Fecha"),
    y=alt.Y("fallecidos_nuevos", title= "Fallecidos"),
    tooltip = "fallecidos_nuevos"
).properties(
    title="Evolución de fallecimientos",
    width=1400,
    height=500
) 

#Casos activos
plot_activos = alt.Chart(casos_activos).mark_line().encode(
    x=alt.X("fecha:T", title= "Fecha"),
    y=alt.Y("casos_activos", title= "Casos activos"),
    tooltip = "casos_activos"
).properties(
    title="Evolución de casos activos",
    width=1400,
    height=500
) 

alt.vconcat(
    plot_confirmados, 
    plot_fallecimientos, 
    plot_activos
).resolve_scale(
    x='shared'
)

**Comentarios:** Comparando los gráficos anteriores podemos decir que los pick, hasta la fecha, de casos confirmados, fallecidos y casos avtivos se vivieron durante junio y julio. En noviembre los registros parecen estabilizarle, a excepción de los fallecidos que siguen oscilando pero que no superan los 100.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:
df= confirmados.groupby(["region","fecha"])["tasa"].mean().reset_index()

In [ ]:
alt.Chart(df).mark_line().encode(
    x=alt.X("fecha:T", title= "Fecha"),
    y=alt.Y("mean(tasa)", title= "Tasa de incidencia promedio"),
    color=alt.Color("region:N", title="Región", scale=alt.Scale(scheme='category20')),
    tooltip = alt.Tooltip("region", title="Región")
).properties(
    title="Evolución tasa promedio por región",
    width=1300,
    height=500
)    

**Comentarios:** En abril e inicio de mayo es dificl concluir algo, de mayo a julio las regiones más afectadas fueron la Metropolitana, Tarapacá y Antofagasta (en ese orden), a finales de julio e inicio de septiembre se suman Arica y Parinacota y Magallanes, respectivamente, a las más afectadas. 

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
trece_abril = confirmados.loc[confirmados["fecha"]=="2020-04-13"]
seis_nov = confirmados.loc[confirmados["fecha"]=="2020-11-06"]

In [ ]:
plot_abril=alt.Chart(trece_abril).mark_circle().encode(
    x=alt.X("casos_confirmados:Q", title= "Casos confirmados"),
    y=alt.Y("tasa:Q", title="Tasa de incidencia"),
    color=alt.Color("region:N", title= "Región", scale=alt.Scale(scheme='category20')),
    tooltip=[alt.Tooltip("region", title="Región")],
    size = alt.Tooltip("poblacion", title="Población")
).properties(
    title="Tasa de incidencia v/s Casos confirmados el 13 de abril de 2020",
    width=1200,
    height=400
)

plot_nov=alt.Chart(seis_nov).mark_circle().encode(
    x=alt.X("casos_confirmados:Q", title= "Casos confirmados"),
    y=alt.Y("tasa:Q", title="Tasa de incidencia"),
    color=alt.Color("region:N", title= "Región", scale=alt.Scale(scheme='category20')),
    tooltip=alt.Tooltip("region", title="Región"),
    size = alt.Tooltip("poblacion", title="Población")
).properties(
    title="Tasa de incidencia v/s Casos confirmados el 06 de noviembre de 2020",
    width=1200,
    height=400
)

display(plot_abril, plot_nov)

**Comentarios:** Comparando los ejes de cada gráfico rapidamente notamos que la tasa de incidencia y los casos confirmados en noviembre son considerablemente mayores, por los colores también podemos decir que en abril La Araucanía, Magallanes y de la Antártica Chilena, Ñuble y la Metropolitana eran las regiones con mayores tasas y casos confirmados sin enbargo, en noviembre la Metropolitana, Antofagasta y Magallanes y de la Antártica Chilena llevan la delantera.
Que el tamaño de los puntos sea proporcional a la población puede ser útil ya que ayuda a dimensionar la relación entre la tasa de incidencia y la población.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:
alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha:T", title= "Fecha"),
    y=alt.Y("casos_activos", title= "Casos activos"),
    color=alt.Color('comuna:N', title="Comuna", scale=alt.Scale(scheme='rainbow')),
    tooltip = "comuna"
).properties(
    title="Evolución de casos activos por comuna",
    width=1200,
    height=400
) 

In [ ]:
for reg in activos.region.unique():
    display(alt.Chart(activos.loc[ activos["region"] == reg]).mark_line(opacity=0.8).encode(
        x=alt.X("fecha:T", title= "Fecha"),
        y=alt.Y("casos_activos:Q", title= "Casos activos"),
        color=alt.Color('comuna:N', title="Comuna", scale=alt.Scale(scheme='rainbow')),
        tooltip=['comuna']
    ).properties(
        title=f"Evolución de casos activos por comuna en la Región {reg}",
        width=1200,
        height=400
    ))

**Comentarios:** 
1. Enfoque Nacional: Lo favorable es que se tiene una visión clara de los registros máximos a nivel país, se puede visualizar cuándo y en qué comuna se registró la cantidad de casos activos maxima, sin embargo el resto de los registros (casos activos < 800) difícilmente se pueden distinguir entre sí.
2. Enfoque regional: En este caso, es más facil visualizar y comparar los registros por región, sobre todo en regiones con pocas comunas, sin embargo en las que hay muchas el problema sigue siendo difícil diferenciar entre registros, además no es conveniente si es que se quiere realizar un análisis nacional.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
region_names = {
    "Araucanía": "La Araucanía",
    "La Araucania": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Biobio": "Biobío",
    "Nuble":"Ñuble",
    "Los Rios": "Los Ríos",
    "Tarapaca": "Tarapacá", 
    "Valparaiso": "Valparaíso"
}

In [ ]:
max_fallecidos = pd.DataFrame(fallecidos.drop(fallecidos[fallecidos["region"]=="Total"].index, axis=0).replace(region_names).groupby("region")["fallecidos"].max()).reset_index()
max_confirmados = pd.DataFrame(confirmados.replace(region_names).groupby("region")["casos_confirmados"].max()).reset_index()
prom_activos = pd.DataFrame(activos.replace(region_names).groupby("region")["casos_activos"].mean()).reset_index()

df=prom_activos.merge(max_confirmados, on = "region").merge(max_fallecidos, on = "region")

In [ ]:
plot_casos_confirmados = alt.Chart(df).mark_circle(opacity=0.7).encode(
    x=alt.X("casos_activos", title="Promedio casos activos" ),
    y=alt.Y("casos_confirmados:Q", title="Máx. casos confirmados (puntos)"),
    color=alt.Color("region:N", title="Comuna", scale=alt.Scale(scheme="category20")),
    tooltip=["region"]
).properties(
    title= "Máximos casos confirmados y fallecidos v/s promedio de casos activos por región",
    width=1000,
    height=500
)
plot_fallecidos=alt.Chart(df).mark_point(shape='square', opacity=0.7).encode(
    x=alt.X("casos_activos", title="Promedio casos activos" ),
    y=alt.Y("fallecidos:Q", title="Máx. fallecidos (cuadrados)"),
    color=alt.Color("region:N", title="Comuna", scale=alt.Scale(scheme="category20")),
    tooltip=["region"]
).properties(
    width=1400,
    height=600
)
plot_casos_confirmados+plot_fallecidos

**Comentarios:** A partir del gráfico anterior podemos decir que la región Metropolitana es la que cuenta con mayor promedio de casos activos, maxima canticadad de casos confirmados y fallecidos. Mientras que en general, podríamos decir que existe cierta relación directa entre el promedio de casos activos y el máximo de casos confirmados, en cambio, no se puede decir lo mismo entre el primero y el máximo de fallecidos.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
shp_filepath = Path().resolve() / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

In [ ]:
type(regiones)

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
casos_geo = (
    confirmados[["region", "region_id"]].drop_duplicates().rename(columns={"region_id":"codigo_region"})
    .merge(confirmados.groupby(["region", "fecha"])["poblacion"].sum().reset_index(), on="region", how="inner")
    .merge(confirmados.groupby(["region", "fecha"])["casos_confirmados"].sum().reset_index(), on=["region","fecha"], how="inner")
    .assign(tasa= lambda x: (x.casos_confirmados/x.poblacion)*100000)
    .replace(region_names)
    .merge(activos.groupby(["codigo_region", "fecha"])["casos_activos"].sum().reset_index(), on=["codigo_region", "fecha"], how="inner")
    .merge(fallecidos.groupby(["region", "fecha"])["fallecidos"].sum().reset_index().replace(region_names), on=["region", "fecha"], how="inner")
    .merge(regiones[["codregion", "geometry"]].rename(columns={"codregion": "codigo_region"}), on="codigo_region", how="inner")
    .fillna(0)
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x['fecha'] == fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(
    options=[("Poblacion", "poblacion"), ("Casos confirmados", "casos_confirmados"),
             ("Tasa", "tasa"), ("Casos activos", "casos_activos"), ("Fallecidos", "fallecidos")], 
    description="Columna",
    continuous_update = False
)

In [ ]:
fecha_widget = widgets.DatePicker(
    description='Fecha',
    continuous_update = False,
    disabled=False
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** Demora demasiado en cargar, no creo que sea muy conveniente utilizarlo para realizar análisis rápido por esta razón.